In [1]:
# import sys  
# sys.path.insert(0, '/Users/arturo/Downloads/')

from function2 import *


In [2]:
import pyspark
from pyspark.sql import SparkSession
import warnings
warnings.filterwarnings('ignore')
from pyspark.sql.window import Window as W
pd.options.display.max_colwidth = 1000
# from pyspark.sql.types import StructType,StructField, StringType, IntegerType 
# from pyspark.sql.types import ArrayType, DoubleType, BooleanType
# from pyspark.sql.functions import col,array_contains



In [3]:
spark = SparkSession.builder.appName('SparkByExamples.com').getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/07/31 18:55:38 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


# Datos

In [4]:
Nombre_Input_entidades = spark.read.csv("DATA_SIC_ENTIDADES_PUBLICAS_AYUDAS_ESP_short1.csv", sep=";", header=True)
Nombre_Input_relaciones = spark.read.csv("DATA_RELACION_ENTIDADES_MAESTRO_short1.csv", sep=";", header=True)
Nombre_Input_investigadores = spark.read.csv("DATA_SIC_AYUDA_ESP_UO_PB_J_MAESTRO_short1.csv", sep=";", header=True)

# Cruce 1

In [5]:
cruce1 = Nombre_Input_investigadores.\
                                    join(Nombre_Input_relaciones,
                                         Nombre_Input_investigadores.Id ==  Nombre_Input_relaciones.Id_NIVEL_0,"left")\
                                    .dropDuplicates(Nombre_Input_investigadores.columns)\
                                    .filter("Id_NIVEL_0 is not null")

In [6]:
cruce1 = cruce1.withColumn("Similitud", 
                           udf_Distance_ratcliff_obershelp(
                               UDF_normalizarTexto(cruce1.NOMBRE_ENTIDAD_NIVEL_1),
                               cruce1.Centro_Norm )
                          ) 

# Nombre_Intermedio_cruce1_entidades

In [7]:
maxId = Nombre_Input_entidades.withColumn("Id", Nombre_Input_entidades.Id.cast('int')).select("Id").rdd.max()[0]

In [8]:
data = cruce1.filter("Similitud <0.875")\
             .withColumn("CIF",lit(None).cast(StringType()))\
             .withColumn("Entidad_Norm",col("Centro_Norm"))\
             .withColumn("Provincia_Entidad",col("Provincia_Centro"))\
             .withColumn("ID_ENTIDAD",lit(None).cast(StringType()))\
             .withColumn("NIF_COD",lit(None).cast(StringType()))\
             .withColumn("ACRONIMO",lit(None).cast(StringType()))\
             .withColumn("NOMBRE_ENTIDAD",lit(None).cast(StringType()))\
             .withColumn("Nombre_Entidad_Mostrar",lit(None).cast(StringType()))\
             .withColumn("TIPO_ENTIDAD_N1_1",lit(None).cast(StringType()))\
             .withColumn("TIPO_ENTIDAD_N2_1",lit(None).cast(StringType()))\
             .withColumn("DIRECCION_POSTAL",lit(None).cast(StringType()))\
             .withColumn("COD_POSTAL",lit(None).cast(StringType()))\
             .withColumn("COD_PROVINCIA",lit(None).cast(StringType()))\
             .withColumn("PROVINCIA",lit(None).cast(StringType()))\
             .withColumn("COD_CCAA",lit(None).cast(StringType()))\
             .withColumn("CCAA",lit(None).cast(StringType()))\
             .withColumn("ENLACE_WEB",lit(None).cast(StringType()))\
             .withColumn("SOMMA",lit(None).cast(StringType()))\
             .withColumn("TIPO_ENTIDAD_REGIONAL",lit(None).cast(StringType()))\
             .withColumn("ESTADO_x",lit(None).cast(StringType()))\
             .withColumn("CodigoInvente",lit(None).cast(StringType()))\
             .withColumn("DenominacionSocial",lit(None).cast(StringType()))\
             .withColumn("FormaJuridica_Codigo",lit(None).cast(StringType()))\
             .withColumn("FormaJuridica_Descripcion",lit(None).cast(StringType()))\
             .withColumn("NIF",lit(None).cast(StringType()))\
             .withColumn("codigoDir3",lit(None).cast(StringType()))\
             .withColumn("codigoOrigen",lit(None).cast(StringType()))\
             .withColumn("Provincia_Codigo",lit(None).cast(StringType()))\
             .withColumn("C_ID_UD_ORGANICA",lit(None).cast(StringType()))\
             .withColumn("C_DNM_UD_ORGANICA",lit(None).cast(StringType()))\
             .withColumn("C_ID_NIVEL_ADMON",lit(None).cast(StringType()))\
             .withColumn("C_ID_TIPO_ENT_PUBLICA",lit(None).cast(StringType()))\
             .withColumn("N_NIVEL_JERARQUICO",lit(None).cast(StringType()))\
             .withColumn("C_ID_DEP_UD_SUPERIOR",lit(None).cast(StringType()))\
             .withColumn("C_DNM_UD_ORGANICA_SUPERIOR",lit(None).cast(StringType()))\
             .withColumn("C_ID_DEP_UD_PRINCIPAL",lit(None).cast(StringType()))\
             .withColumn("C_DNM_UD_ORGANICA_PRINCIPAL",lit(None).cast(StringType()))\
             .withColumn("B_SW_DEP_EDP_PRINCIPAL",lit(None).cast(StringType()))\
             .withColumn("C_ID_DEP_EDP_PRINCIPAL",lit(None).cast(StringType()))\
             .withColumn("C_DNM_UD_ORGANICA_EDP_PRINCIPAL",lit(None).cast(StringType()))\
             .withColumn("C_ID_ESTADO",lit(None).cast(StringType()))\
             .withColumn("D_VIG_ALTA_OFICIAL",lit(None).cast(StringType()))\
             .withColumn("NIF_CIF",lit(None).cast(StringType()))\
             .withColumn("C_ID_AMB_PROVINCIA",lit(None).cast(StringType()))\
             .withColumn("C_DESC_PROV",lit(None).cast(StringType()))\
             .withColumn("CONTACTOS",lit(None).cast(StringType()))\
             .withColumn("List_Entidad_Norm",col("Centro_Norm"))\
             .withColumn("List_Provincia_Entidad",col("Provincia_Centro"))\
             .withColumn("List_CIF",lit(None).cast(StringType()))

In [9]:
data_count = data.count()

In [10]:
#data = data.dropDuplicates(["Entidad_Norm", "Provincia_Entidad"])
data = data.withColumnRenamed("Id", "IdOld")

In [11]:
Index = [maxId + i for i in range(1, data_count+1)]
Index = spark.createDataFrame(Index, IntegerType())
Index = Index.withColumnRenamed("value", "Id")

In [12]:
windowSpec = W.orderBy("Dummy")

Index = Index.withColumn("Dummy", monotonically_increasing_id())
Index = Index.withColumn("Dummy", F.row_number().over(windowSpec))

data = data.withColumn("Dummy", monotonically_increasing_id())
data = data.withColumn("Dummy", F.row_number().over(windowSpec))

data = data.join(Index, "Dummy", "outer").drop("Dummy")

In [13]:
Nombre_Intermedio_cruce1_entidades = data.select(Nombre_Input_entidades.columns)

In [14]:
# eliminar duplicados

In [15]:
#Nombre_Intermedio_cruce1_entidades.write.option("header",True).mode('overwrite').format("csv").save("Nombre_Intermedio_cruce1_entidades/csv")

In [16]:
#Nombre_Intermedio_cruce1_entidades.write.format("parquet").mode('overwrite').save("Nombre_Intermedio_cruce1_entidades/parquet")

# Nombre_Output_investigadores

In [17]:
data_2 = cruce1.filter("Similitud >=0.875")\
        .withColumn("Final_Score", col("Similitud"))\
        .withColumn("Id_Centro", col("Id_NIVEL_1"))\
        .withColumn("Centro_Nombre_match", col("NOMBRE_ENTIDAD_NIVEL_1"))\
        .withColumn("Centro_Provincia_match", col("COD_PROVINCIA_NIVEL_1"))\
        .withColumn("Similitud_Centro", col("Similitud"))\
        .select(Nombre_Input_investigadores.columns + 
                ["Final_Score","Id_Centro", "Centro_Nombre_match", "Centro_Provincia_match", "Similitud_Centro"])


In [18]:
data_3 = data.withColumn("Final_Score", col("Similitud"))\
        .withColumn("Id_Centro", col("Id"))\
        .withColumn("Centro_Nombre_match", col("NOMBRE_ENTIDAD_NIVEL_1"))\
        .withColumn("Centro_Provincia_match", col("COD_PROVINCIA_NIVEL_1"))\
        .withColumn("Similitud_Centro", col("Similitud"))\
        .select(Nombre_Input_investigadores.columns + 
                ["Final_Score","Id_Centro", "Centro_Nombre_match", "Centro_Provincia_match", "Similitud_Centro"])


In [19]:
Nombre_Output_investigadores = data_2.union(data_3)

In [20]:
#result.write.option("header",True).mode('overwrite').format("csv").save("Nombre_Output_investigadores/csv")

In [21]:
#result.write.format("parquet").mode('overwrite').save("Nombre_Output_investigadores/parquet")

# Nombre_Intermedio_cruce1_relaciones

In [22]:
Nombre_Intermedio_cruce1_relaciones = data.select(["Id", "IdOld", "Entidad", "Centro","Centro_Norm", "Provincia_Entidad", "Provincia_Centro","CIF","Entidad_Norm"])

In [23]:
#Eliminamos duplicados

l = get_matching_entities_by_elasticSearch(
                                        Nombre_Input_investigadores, 
                                        "Entidad_Norm", 
                                        ["Entidad_Norm"],
                                        Nombre_Input_investigadores, 
                                        "Centro_Norm",
                                        ["Centro_Norm"],
                                        'index_name',
                                        10)

l = get_matching_by_elasticSearch_Distance_ratcliff_obershelp(spark,l)
duplicados = list(l.filter("final_score >= 0.875").select("Entidad_Norm").dropDuplicates().toPandas().Entidad_Norm)
Nombre_Intermedio_cruce1_relaciones = Nombre_Intermedio_cruce1_relaciones.filter(~Nombre_Intermedio_cruce1_relaciones.Entidad_Norm.isin(duplicados))


100%|████████████████████████████████████████| 970/970 [00:05<00:00, 185.69it/s]


22/07/31 18:56:05 WARN StopWordsRemover: Default locale set was [en_MX]; however, it was not found in available locales in JVM, falling back to en_US locale. Set param `locale` in order to respect another locale.
22/07/31 18:56:05 WARN StopWordsRemover: Default locale set was [en_MX]; however, it was not found in available locales in JVM, falling back to en_US locale. Set param `locale` in order to respect another locale.


In [24]:
Nombre_Intermedio_cruce1_relaciones = Nombre_Intermedio_cruce1_relaciones.withColumn("Id_NIVEL_1", col("Id"))\
      .withColumn("Id_NIVEL_0", col("IdOld"))\
      .withColumn("NOMBRE_ENTIDAD_NIVEL_0", col("Entidad"))\
      .withColumn("NOMBRE_ENTIDAD_NIVEL_1", col("Centro"))\
      .withColumn("COD_PROVINCIA_NIVEL_0", col("Provincia_Entidad"))\
      .withColumn("COD_PROVINCIA_NIVEL_1", col("Provincia_Centro"))\
      .withColumn("NIF_COD_NIVEL_0", col("CIF"))\
      .withColumn("Jerarquia", lit("Orgánica Directa").cast(StringType()))\
      .withColumn("ACRONIMO_NIVEL_0",lit(None).cast(StringType()))\
      .withColumn("ACRONIMO_NIVEL_1",lit(None).cast(StringType()))\
      .withColumn("CCAA_NIVEL_0",lit(None).cast(StringType()))\
      .withColumn("CCAA_NIVEL_1",lit(None).cast(StringType()))\
      .withColumn("COD_CCAA_NIVEL_0",lit(None).cast(StringType()))\
      .withColumn("COD_CCAA_NIVEL_1",lit(None).cast(StringType()))\
      .withColumn("COD_POSTAL_NIVEL_0",lit(None).cast(StringType()))\
      .withColumn("COD_POSTAL_NIVEL_1",lit(None).cast(StringType()))\
      .withColumn("DIRECCION_POSTAL_NIVEL_0",lit(None).cast(StringType()))\
      .withColumn("DIRECCION_POSTAL_NIVEL_1",lit(None).cast(StringType()))\
      .withColumn("ENLACE_WEB_NIVEL_0",lit(None).cast(StringType()))\
      .withColumn("ENLACE_WEB_NIVEL_1",lit(None).cast(StringType()))\
      .withColumn("ESTADO_x_NIVEL_0",lit(None).cast(StringType()))\
      .withColumn("ENLACE_WEB_NIVEL_1",lit(None).cast(StringType()))\
      .withColumn("ESTADO_x_NIVEL_0",lit(None).cast(StringType()))\
      .withColumn("ESTADO_x_NIVEL_1",lit(None).cast(StringType()))\
      .withColumn("ID_ENTIDAD_NIVEL_0",lit(None).cast(StringType()))\
      .withColumn("ID_ENTIDAD_NIVEL_1",lit(None).cast(StringType()))\
      .withColumn("NIF_COD_NIVEL_1",lit(None).cast(StringType()))\
      .withColumn("Nombre_Entidad_Mostrar_NIVEL_0",lit(None).cast(StringType()))\
      .withColumn("Nombre_Entidad_Mostrar_NIVEL_1",lit(None).cast(StringType()))\
      .withColumn("PROVINCIA_NIVEL_0",lit(None).cast(StringType()))\
      .withColumn("PROVINCIA_NIVEL_1",lit(None).cast(StringType()))\
      .withColumn("SOMMA_NIVEL_0",lit(None).cast(StringType()))\
      .withColumn("SOMMA_NIVEL_1",lit(None).cast(StringType()))\
      .withColumn("TIPO_ENTIDAD_N1_1_NIVEL_0",lit(None).cast(StringType()))\
      .withColumn("TIPO_ENTIDAD_N1_1_NIVEL_1",lit(None).cast(StringType()))\
      .withColumn("TIPO_ENTIDAD_N2_1_NIVEL_0",lit(None).cast(StringType()))\
      .withColumn("TIPO_ENTIDAD_N2_1_NIVEL_1",lit(None).cast(StringType()))\
      .withColumn("TIPO_ENTIDAD_REGIONAL_NIVEL_0",lit(None).cast(StringType()))\
      .withColumn("TIPO_ENTIDAD_REGIONAL_NIVEL_1",lit(None).cast(StringType()))\
    

In [25]:
Nombre_Intermedio_cruce1_relaciones = Nombre_Intermedio_cruce1_relaciones.select(Nombre_Input_relaciones.columns)

In [26]:

#Nombre_Intermedio_cruce1_relaciones.write.option("header",True).mode('overwrite').format("csv").save("Nombre_Intermedio_cruce1_relaciones/csv")

In [27]:
#Nombre_Intermedio_cruce1_relaciones.write.option("header",True).mode('overwrite').format("csv").save("Nombre_Intermedio_cruce1_relaciones/parquet")

# Agregacion vertical de relaciones

In [28]:
Nombre_Output_relaciones = Nombre_Input_relaciones.union(Nombre_Intermedio_cruce1_relaciones)

In [31]:
#Nombre_Intermedio_cruce1_relaciones.write.option("header",True).mode('overwrite').format("csv").save("Nombre_Output_relaciones/csv")

In [32]:
#Nombre_Intermedio_cruce1_relaciones.write.mode('overwrite').format("paequet").save("Nombre_Output_relaciones/parquet")

# If flag_invente_dir3 == True

# Nombre_Input_invente

In [22]:
# data invente
Nombre_Input_invente = pd.read_excel('Analisis_entidades_invente_15-02-2022.xlsx')
for i in Nombre_Input_invente.columns:
    Nombre_Input_invente[i] = Nombre_Input_invente[i].astype(str)



new_header = Nombre_Input_invente.iloc[0] 
# #grab the first row for the header
Nombre_Input_invente = Nombre_Input_invente[1:] #take the data less the header row
Nombre_Input_invente.columns = new_header #set the header row as the df header

Nombre_Input_invente = Nombre_Input_invente[["DenominacionSocial", "Provincia_Codigo", "NIF",
                                                "CodigoInvente", "FormaJuridica_Codigo", 
                                                "FormaJuridica_Descripcion", "codigoDir3","codigoOrigen",
                                                "Provincia_Codigo"]]

Nombre_Input_invente = Nombre_Input_invente.loc[:,~Nombre_Input_invente.columns.duplicated()]

Nombre_Input_invente = spark.createDataFrame(Nombre_Input_invente.fillna('')) 


Nombre_Input_invente = Nombre_Input_invente.withColumn("DenominacionSocial_Norm", UDF_normalizarTexto(Nombre_Input_invente.DenominacionSocial))


# Nombre_Intermedio_cruce2_invente

In [23]:
l = get_matching_entities_by_elasticSearch(
                                        Nombre_Intermedio_cruce1_entidades, 
                                        "Entidad_Norm", 
                                        ["Entidad_Norm"],
                                        Nombre_Input_invente, 
                                        "DenominacionSocial_Norm",
                                        Nombre_Input_invente.columns,
                                        'index_name',
                                        1)

22/07/31 18:38:21 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/07/31 18:38:21 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/07/31 18:38:21 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/07/31 18:38:21 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/07/31 18:38:21 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.
22/07/31 18:38:21 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradat

100%|████████████████████████████████████████| 727/727 [00:03<00:00, 190.29it/s]


In [24]:
l = get_matching_by_elasticSearch_Distance_ratcliff_obershelp(spark,l).filter("final_score >= 0.875")

22/07/31 18:38:33 WARN StopWordsRemover: Default locale set was [en_MX]; however, it was not found in available locales in JVM, falling back to en_US locale. Set param `locale` in order to respect another locale.
22/07/31 18:38:33 WARN StopWordsRemover: Default locale set was [en_MX]; however, it was not found in available locales in JVM, falling back to en_US locale. Set param `locale` in order to respect another locale.


In [25]:
l = l.sort(l.final_score.desc()).dropDuplicates(subset = ['Entidad_Norm']).select("Entidad_Norm","DenominacionSocial", "NIF",
                                                                                "CodigoInvente", "FormaJuridica_Codigo", 
                                                                                "FormaJuridica_Descripcion", "codigoDir3","codigoOrigen",
                                                                                "Provincia_Codigo", "final_score","DenominacionSocial_Norm")


In [26]:
Nombre_Intermedio_cruce2_invente = Nombre_Intermedio_cruce1_entidades.drop("CodigoInvente","DenominacionSocial", "FormaJuridica_Codigo", 
                                                                           "FormaJuridica_Descripcion", "NIF", "codigoDir3", "codigoOrigen",
                                                                            "Provincia_Codigo", "Provincia_Entidad", "CIF")





In [27]:
Nombre_Intermedio_cruce2_invente = Nombre_Intermedio_cruce2_invente.join(l,
                                      Nombre_Intermedio_cruce2_invente.Entidad_Norm ==  l.Entidad_Norm,"left")                                 

     

In [28]:
Nombre_Intermedio_cruce2_invente1 = Nombre_Intermedio_cruce2_invente.filter("final_score is not NULL").drop("Entidad_Norm")
Nombre_Intermedio_cruce2_invente2 = Nombre_Intermedio_cruce2_invente.filter("final_score is  NULL")

In [29]:
Nombre_Intermedio_cruce2_invente1 = Nombre_Intermedio_cruce2_invente1.withColumn("Entidad_Norm", col("DenominacionSocial_Norm"))\
                                                                    .withColumn("Provincia_Entidad", col("Provincia_Codigo"))\
                                                                    .withColumn("CIF", col("NIF"))\
                                                                    .withColumn("Similitud_invente", col("final_score"))

In [30]:
Nombre_Intermedio_cruce2_invente1 = Nombre_Intermedio_cruce2_invente1.select(Nombre_Intermedio_cruce1_entidades.columns+["Similitud_invente"])


In [31]:
Nombre_Intermedio_cruce2_invente = Nombre_Intermedio_cruce2_invente1.union(Nombre_Intermedio_cruce2_invente2)

In [43]:
#Nombre_Intermedio_cruce2_invente.write.option("header",True).mode('overwrite').format("csv").save("Nombre_Intermedio_cruce2_invente/csv")



In [44]:
#Nombre_Intermedio_cruce2_invente.write.mode('overwrite').format("paequet").save("Nombre_Intermedio_cruce2_invente/parquet")



# Nombre_Intermedio_proceso_dir3_entidades

In [32]:
Nombre_Input_dir3 = spark.read.csv("DIR3_concatenado_AGE_CCAA_EELL_Universidades.csv", sep=";", header=True).drop("Unnamed: 0")
Nombre_Input_dir3 = Nombre_Input_dir3.withColumn("C_DNM_UD_ORGANICA_Norm", UDF_normalizarTexto(Nombre_Input_dir3.C_DNM_UD_ORGANICA))


In [33]:
l = get_matching_entities_by_elasticSearch(
                                        Nombre_Intermedio_cruce2_invente, 
                                        "Entidad_Norm", 
                                        ["Entidad_Norm"],
                                        Nombre_Input_dir3, 
                                        "C_DNM_UD_ORGANICA_Norm",
                                        Nombre_Input_dir3.columns,
                                        'index_name',
                                        10)

l = get_matching_by_elasticSearch_Distance_ratcliff_obershelp(spark,l).filter("final_score >= 0.875")\
                               .dropDuplicates(subset = ['Entidad_Norm', "C_DNM_UD_ORGANICA_Norm"])
                                                                    

22/07/31 18:38:47 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/07/31 18:38:47 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/07/31 18:38:47 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/07/31 18:38:47 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/07/31 18:38:47 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/07/31 18:38:47 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/07/31 1

22/07/31 18:38:52 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/07/31 18:38:52 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/07/31 18:38:52 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/07/31 18:38:52 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/07/31 18:38:52 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/07/31 18:38:52 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/07/31 1

100%|████████████████████████████████████████| 727/727 [00:01<00:00, 455.94it/s]


22/07/31 18:40:32 WARN StopWordsRemover: Default locale set was [en_MX]; however, it was not found in available locales in JVM, falling back to en_US locale. Set param `locale` in order to respect another locale.
22/07/31 18:40:32 WARN StopWordsRemover: Default locale set was [en_MX]; however, it was not found in available locales in JVM, falling back to en_US locale. Set param `locale` in order to respect another locale.


In [34]:
unique = l.selectExpr(
              '*', 
              'count(*) over (partition by Entidad_Norm) as cnt'
            ).filter(F.col('cnt') == 1).drop('cnt')



In [35]:
duplicated = l.join(l.groupBy('Entidad_Norm')\
                  .count().where('count = 1').drop('count'),
                on=['Entidad_Norm'],
                how='left_anti')

In [36]:
duplicated1 = duplicated.selectExpr(
              '*', 
              'count(*) over (partition by Entidad_Norm, N_NIVEL_JERARQUICO) as cnt'
            ).filter(F.col('cnt') == 1).drop('cnt')

duplicated1 = duplicated1.sort(duplicated1.N_NIVEL_JERARQUICO.asc())\
                            .dropDuplicates(subset = ['Entidad_Norm'])

In [37]:
duplicated2 = l.join(l.groupBy('Entidad_Norm', 'N_NIVEL_JERARQUICO')\
                  .count().where('count = 1').drop('count'),
                on=['Entidad_Norm','N_NIVEL_JERARQUICO'],
                how='left_anti')


duplicated2 = duplicated2.withColumn("Similitud", 
                           udf_Distance_ratcliff_obershelp(
                               UDF_normalizarTexto(duplicated2.C_DNM_UD_ORGANICA_SUPERIOR),
                               duplicated2.Entidad_Norm )
                          ) 



duplicated2 = duplicated2.sort(duplicated2.Similitud.desc())\
                            .dropDuplicates(subset = ['Entidad_Norm']).drop("Similitud")

In [38]:
match = unique.union(duplicated1).union(duplicated2)\
               .select(['Entidad_Norm',"final_score"] + Nombre_Input_dir3.columns)\
               .withColumnRenamed("Entidad_Norm","Entidad_Norm1")


In [56]:
Nombre_Intermedio_proceso_dir3_entidades = Nombre_Intermedio_cruce2_invente\
                                            .drop('C_DNM_UD_ORGANICA_SUPERIOR', 'C_ID_UD_ORGANICA', 'C_ID_TIPO_ENT_PUBLICA',
                                                  'C_DESC_PROV', 'C_ID_DEP_EDP_PRINCIPAL', 'C_ID_ESTADO', 'D_VIG_ALTA_OFICIAL', 
                                                  'CONTACTOS', 'NIF_CIF', 'C_ID_NIVEL_ADMON', 'C_DNM_UD_ORGANICA_EDP_PRINCIPAL', 
                                                  'C_ID_DEP_UD_PRINCIPAL', 'N_NIVEL_JERARQUICO', 'C_DNM_UD_ORGANICA', 
                                                  'C_DNM_UD_ORGANICA_PRINCIPAL', 'B_SW_DEP_EDP_PRINCIPAL', 'C_ID_AMB_PROVINCIA',
                                                  'C_ID_DEP_UD_SUPERIOR')


In [57]:
Nombre_Intermedio_proceso_dir3_entidades = Nombre_Intermedio_proceso_dir3_entidades.join(match,
                                      Nombre_Intermedio_proceso_dir3_entidades.Entidad_Norm ==  match.Entidad_Norm1,"left")     



In [58]:
Nombre_Intermedio_proceso_dir3_entidades1 = Nombre_Intermedio_proceso_dir3_entidades.filter("final_score is not NULL").drop("Entidad_Norm")

Nombre_Intermedio_proceso_dir3_entidades2 = Nombre_Intermedio_proceso_dir3_entidades.filter("final_score is  NULL").drop("Entidad_Norm1")

Nombre_Intermedio_proceso_dir3_entidades2 = Nombre_Intermedio_proceso_dir3_entidades2\
                                                .withColumn("Similitud_Dir3",lit(None).cast(FloatType()))

In [59]:
Nombre_Intermedio_proceso_dir3_entidades1 = Nombre_Intermedio_proceso_dir3_entidades1\
                                .withColumn("Entidad_Norm", col("C_DNM_UD_ORGANICA"))\
                                .withColumn("Provincia_Entidad", col("C_DESC_PROV"))\
                                .withColumn("CIF", col("NIF_CIF"))\
                                .withColumn("Similitud_Dir3", col("final_score"))\
                                .drop("Entidad_Norm1")




In [60]:
Nombre_Intermedio_cruce3_dir3 = Nombre_Intermedio_proceso_dir3_entidades1\
                                            .union(Nombre_Intermedio_proceso_dir3_entidades2)\
                                            .drop("C_DNM_UD_ORGANICA_Norm", "final_score")






In [ ]:
#Nombre_Intermedio_cruce3_dir3.write.option("header",True).mode('overwrite').format("csv").save("Nombre_Intermedio_cruce3_dir3/csv")


In [ ]:
#Nombre_Intermedio_cruce3_dir3.write.mode('overwrite').format("paequet").save("Nombre_Intermedio_cruce3_dir3/parquet")


# Inclusion vertical

In [61]:
Nombre_Output_entidades = Nombre_Intermedio_cruce3_dir3.drop("Similitud_Invente", "Similitud_Dir3")

In [62]:
Nombre_Output_entidades = Nombre_Input_entidades\
                                            .union(Nombre_Output_entidades)

In [ ]:
#Nombre_Output_entidades.write.option("header",True).mode('overwrite').format("csv").save("Nombre_Output_entidades/csv")


In [ ]:
#Nombre_Output_entidades.write.mode('overwrite').format("parquet").save("Nombre_Output_entidades/parquet")


# If flag_invente_dir3 == False

In [30]:
Nombre_Output_entidades = Nombre_Input_entidades\
                                            .union(Nombre_Intermedio_cruce1_entidades)

In [ ]:
#Nombre_Output_entidades.write.option("header",True).mode('overwrite').format("csv").save("Nombre_Output_entidades/csv")


In [ ]:
#Nombre_Output_entidades.write.mode('overwrite').format("parquet").save("Nombre_Output_entidades/parquet")
